Queremos hacer un estudio de mercado del sector del automóvil en España, cómo han evolucionado las ventas a lo largo del tiempo.

de aquí saco las matriculaciones por año: https://datosmacro.expansion.com/negocios/matriculaciones-vehiculos/espana?anio=2021

de aquí saco una API con los modelos = https://public.opendatasoft.com/explore/dataset/all-vehicles-model/api/

API para modelos por años: https://rapidapi.com/koukoulina001/api/cars21/playground/apiendpoint_23fe4099-3684-4e75-8a43-9c7a880e34ba

Aquí info sobre modelos: https://anfac.com/cifras-clave/matriculaciones-turismos-y-todoterreno/

Coches más vendidos por años: https://www.autofacil.es/industria/coches-mas-vendidos-historia-espana/178791.html

API Wallapop: http://api.wallapop.com/api/v3/general/search


In [57]:
import requests
import pandas as pd
import sys
sys.path.append("../")
import src
from src import soporte

In [58]:
#Test API wallapop: FUNCIONA!!!!!!!!

url = "http://api.wallapop.com/api/v3/general/search"

headers = {'Accept': '*/*', 'User-Agent': 'Wget/1.21.4',

'Accept-Encoding': 'identity', 'X-DeviceOS': '0' }

params = {"search_objects":[],
          # "from":0,
          # "to":10,
          "keywords":"Dacia",
          "min_sale_price" : 1000,
          "max_sale_price" : 10000,
          "order":"most_relevance",
          "search_point":{"latitude":40.450381,"longitude":-3.518064},
          "type": "cars_search_cars",
          "category_id" : 100,
          # "brand": "Dacia",
          # "model": "Dokker",
          "max_km": 100000
        #   "spellcheck":null
        }

res = requests.get(url, headers = headers, params = params)
res.status_code
dc_wallapop = res.json()

In [59]:
#Sacamos los precios de la API
lista_precios = []
for i in dc_wallapop["search_objects"]:
    lista_precios.append(i["price"])

#Sacamos el nombre
lista_nombres = []
for i in dc_wallapop["search_objects"]:
    lista_nombres.append(i["title"])

#Sacamos el link
lista_links = []
for i in dc_wallapop["search_objects"]:
    lista_links.append("es.wallapop.com/item/" + i["web_slug"])

#Sacamos localizacion
lista_localiz = []
for i in dc_wallapop["search_objects"]:
    lista_localiz.append(list(i["location"].values())[0])

#Sacamos Código postal:
lista_postal = []
for i in dc_wallapop["search_objects"]:
    lista_postal.append(list(i["location"].values())[1])

#Sacamos país:
lista_pais = []
for i in dc_wallapop["search_objects"]:
    lista_pais.append(list(i["location"].values())[-1])




In [60]:
#Sacamos el df con todos los modelos detallados y el precio mediano 
df_wallapop = pd.DataFrame(list(zip(lista_nombres,lista_precios,lista_localiz,lista_postal,lista_links)), 
                           columns = ["Modelo","Precio","Localización","Código_postal","Link_wallapop"])

valor_mediano = int(df_wallapop["Precio"].median())

# #Calculamos el precio que más se acerca al precio mediano
# precio_wallapop = soporte.acercarse(valor_mediano,lista_precios)
# print(f"el precio del modelo que más se acerca al precio mediano es: {precio_wallapop}")

valor_mediano


7355

In [61]:
def numero_mas_cercano(valor, lista_numeros):
    # Calcula la diferencia absoluta y encuentra el número más cercano
    numero_cercano = min(lista_numeros, key=lambda x: abs(x - valor))
    return numero_cercano
    
numerito = numero_mas_cercano(valor_mediano,lista_precios)

In [63]:
filtro = df_wallapop[df_wallapop["Precio"] == numerito]
    
filtro

,Modelo,Precio,Localización,Código_postal,Link_wallapop
10,DACIA Dokker Dokker Van Essential 1.6 75 kW (1...,7355.0,Mercado Parque Corredor del Henares,28851,es.wallapop.com/item/dacia-dokker-dokker-van-e...
12,DACIA Dokker Dokker Van Essential 1.6 102CV GLP,7355.0,Mercado Parque Corredor del Henares,28851,es.wallapop.com/item/dacia-dokker-dokker-van-e...
13,DACIA Dokker Dokker Van Essential 1.6 102CV GLP,7355.0,Mercado Parque Corredor del Henares,28851,es.wallapop.com/item/dacia-dokker-dokker-van-e...
24,Dacia Dokker 2020,7355.0,Ciudad Deportiva del Jarama,28709,es.wallapop.com/item/dacia-dokker-2020-1020880011
